# ▼ 크롤링 시작하기

## 1 클래스 정의

In [96]:
class MovieAPI():
    def __init__(self):
        self.index = []
        self.title_li = []
        self.link_li = []
        self.image_li = []
        self.year_li = []
        self.dir_li = []
        self.actor_li = []
        self.rating_li = []
        self.error_index = []
        self.start = 1

    # ▼ 리스트까지만 반환하는 함수
    def get_movie(self, index_list, movie_list, dir_list, year_list, client_id, client_secret):
        self.index_list = index_list
        self.movie_list = movie_list
        self.dir_list = dir_list
        self.year_list = year_list
        self.client_id = client_id
        self.ciient_secret = client_secret
        
        import urllib.request
        import xmltodict
        import pandas as pd
        import time
        import re
        
        for i, movie_o in enumerate(movie_list):
            movie = movie_o.replace('(애니)','').replace('(다큐)','').replace('감독판','').replace('확장판','').replace('X2','엑스 2').replace('(6/45)','')
            movie = re.sub('[^가-힣A-Za-z0-9\s-]', '', movie)   # 한글, 영어, 숫자, 공백, - 빼고 지우기
            encText = urllib.parse.quote(movie)
            params = f'"&display=100&start={self.start}&yearfrom={str(int(year_list[i])-1)}&yearto={str(int(year_list[i]))}'
            url = 'https://openapi.naver.com/v1/search/movie.xml?query="' + encText  + params
            time.sleep(0.5)
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            response = urllib.request.urlopen(request)
            rescode = response.getcode()
            if(rescode==200):
                response_body = response.read()
                dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
                search_li = dict_data['rss']['channel']
                if int(search_li['total']) != 0:
                    # 검색 결과 있는 경우 
                    search_li = search_li['item']
                    if type(search_li) == list:
                        # 검색 결과가 여러개인 경우
                        for j in range(len(search_li)):
                            mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','').replace('감독판','').replace('확장판','').replace('X2','엑스 2').replace('(6/45)','')
                            mv_title = re.sub('[^가-힣A-Za-z0-9\s-]', '', mv_title)
                            mv_rating = search_li[j]['userRating']
                            mv_dir = search_li[j]['director']
                            mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                            mv_actor = search_li[j]['actor']
                            mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                            mv_year = search_li[j]['pubDate']
                            if mv_title.replace(' ','') == movie.replace(' ','') and mv_rating != None and mv_dir != None and mv_actor != None:
                                # 영화명 같고, 평점-감독-배우 있으면,
                                if dir_list[i] in mv_dir:
                                    # 감독명이 API에서 찾은 리스트 있으면
                                    self.index.append(index_list[i])
                                    self.title_li.append(movie_o)
                                    self.link_li.append(search_li[j]['link'])
                                    self.image_li.append(search_li[j]['image'])
                                    self.year_li.append(mv_year)
                                    self.dir_li.append(mv_dir)
                                    self.actor_li.append(mv_actor)
                                    self.rating_li.append(mv_rating)
                                    break   # 뒤에 더 탐색할 필요가 없음
                                elif self.index != [] and self.index[-1] != i:
                                    # 감독명이 일치하지 않은 경우, 아직 append 안됐을 때
                                    if int(dict_data['rss']['channel']['total']) <= 100:
                                        # 이때, 100개 미만이면 없는 걸로 알고 다음으로
                                        self.error_index.append([index_list[i], '3rd'])
                                        break
                                    else:
                                        # 100개가 넘으면 재귀, start 값이 페이지 넘어가면 break
                                        self.start += 1
                                        if self.start == (int(dict_data['rss']['channel']['total'])//100)+2:
                                            self.start = 1
                                            break
                                        MovieAPI.get_movie(self, [index_list[i]], [movie], [mv_dir], [search_li[j]['pubDate']], client_id, client_secret)
                                        break
                                else:   # 감독도 일치하지 않고, 평점도 0.00이고
                                    self.error_index.append([index_list[i], '2nd'])
                                    break
                            elif mv_rating != None and mv_dir != None and mv_actor != None:
                                # 영화명 같지 않지만 평점-감독-배우 있는 경우
                                if dir_list[i] in mv_dir:
                                    # 감독명이 API에서 찾은 리스트 있으면
                                    self.index.append(index_list[i])
                                    self.title_li.append(movie_o)
                                    self.link_li.append(search_li[j]['link'])
                                    self.image_li.append(search_li[j]['image'])
                                    self.year_li.append(mv_year)
                                    self.dir_li.append(mv_dir)
                                    self.actor_li.append(mv_actor)
                                    self.rating_li.append(mv_rating)
                                    break   # 뒤에 더 탐색할 필요가 없음
                                elif self.index != [] and self.index[-1] != i and mv_rating != '0.00':
                                    # 감독명이 일치하지 않은 경우, 아직 append 안됐을 때, 일단 있는 거 넣어라
                                    if int(dict_data['rss']['channel']['total']) <= 100:
                                        # 이때, 100개 미만이면 없는 걸로 알고 다음으로
                                        self.error_index.append([index_list[i], '3rd'])
                                        break
                                    else:
                                        # 100개가 넘으면 재귀, start 값이 페이지 넘어가면 break
                                        self.start += 1
                                        if self.start == (int(dict_data['rss']['channel']['total'])//100)+2:
                                            self.start = 1
                                            break
                                        else:
                                            MovieAPI.get_movie(self, [index_list[i]], [movie_o], [mv_dir], [search_li[j]['pubDate']], client_id, client_secret)
                                            break
                                else:   # 감독도 일치하지 않는 경우
                                    self.error_index.append([index_list[i], '2nd'])
                                    break
                            else:       # 영화명도 다르고 평점-감독-배우 None
                                self.error_index.append([index_list[i], '1st'])
                                break
                    elif type(search_li) == dict:
                        # 검색 결과가 하나인 경우
                        mv_title = search_li['title'].replace('</b>','').replace('<b>','')
                        mv_dir = search_li['director']
                        mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                        mv_actor = search_li['actor']
                        mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

                        self.index.append(index_list[i])
                        self.title_li.append(movie_o)
                        self.link_li.append(search_li['link'])
                        self.image_li.append(search_li['image'])
                        self.year_li.append(search_li['pubDate'])
                        self.dir_li.append(mv_dir)
                        self.actor_li.append(mv_actor)
                        self.rating_li.append(search_li['userRating'])
                    else:
                        # 아마 없을듯
                        self.error_index.append([index_list[i], 'list/dict 분류 오류'])
                else: 
                    # 검색 안되는 경우
                    self.error_index.append([index_list[i], '검색 불가'])
            else:
                print("Error Code:" + rescode)
            if (i != 0 and i % 10 == 0) or i == (len(movie_list)-1):
                print(f'{index_list[i]}번째 완료')
            if i != 0 and i % 500 == 0:    # 500번마다 savepoint로 저장해두기 
                pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose().to_csv(f'./files/savepoint{index_list[i]}_df2_utf-8.csv', encoding='utf-8')
                print('savepoint', index_list[i])
        # for문 끝나면
        pd.DataFrame(self.error_index).to_csv(f'./files/error_list2_utf8.csv', encoding='utf-8')
        
        return [self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]
                
    # ▼ get_movie 실행 및 저장 함수
    def mk_movie_df(self, index_list, movie_list, dir_list, year_list, client_id, client_secret):
        import pandas as pd
        
        li_to_df = MovieAPI.get_movie(self, index_list, movie_list, dir_list, year_list, client_id, client_secret)
        movie_df = pd.DataFrame(li_to_df).transpose()
        movie_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '연도', '감독', '출연', '평점']

        # 결과 데이터 중복 제거
        df_drop_dp = movie_df.drop_duplicates(['순번'], keep='last')
        df_drop_dp

        # 인덱스 설정
        movie_df = movie_df.set_index('순번')
        df_drop_dp = df_drop_dp.set_index('순번')

        # 결과 데이터와 중복 제거 데이터 저장
        movie_df.to_csv('./files/naver_api2_utf-8.csv', encoding='utf-8')
        df_drop_dp.to_csv('./files/naver_api2_drop_dupl_utf-8.csv', encoding='utf-8')

        return movie_df, df_drop_dp


## 2 리스트 불러오기

### 2-1 연도 컬럼 만들기(연도 컬럼 없는 경우) 

In [25]:
import pandas as pd

movie_df = pd.read_csv('movie_resize2.csv', encoding='utf-8')
year_df = pd.DataFrame()
for row in movie_df['개봉일']:
    year_df = pd.concat([year_df, pd.DataFrame([row.split('-')[0]])])

year_df.columns = ['연도']

year_df.index = range(4642)

movie_df = pd.read_csv('movie_resize2.csv', encoding='utf-8')
movie_df = pd.concat([movie_df, year_df], axis=1)

movie_df.to_csv('movie_resize2_year_utf8.csv', encoding='utf-8')

,연도
0,2014
0,2019
0,2017
0,2014
0,2019
...,...
0,2015
0,2011
0,2010
0,2019


### 2-2 파일 불러오기
DataFrame 불러와서 '인덱스', '영화명', '감독', '연도' 리스트로 각자 넣어주기

In [16]:
import pandas as pd

# 리스트 불러오기
movie_df = pd.read_csv('movie_resize2_year_utf8.csv', encoding='utf-8', index_col=0)
display(movie_df[:6])
index_list = list(movie_df.index)
movie_list = movie_df['영화명'].to_list()
dir_list = movie_df['감독'].to_list()
year_list = movie_df['연도'].to_list()

print('len =',len(index_list,), index_list[:10])
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])
print('len =',len(year_list), year_list[:10])


,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
0,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,"33,121,225,810","4,163,666",사극,15세관람가,일반영화,2014
1,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,"31,858,660,536","3,638,287",코미디,15세관람가,일반영화,2019
2,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,"27,530,825,087","3,346,172",판타지,12세관람가,일반영화,2017
3,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,"25,842,519,330","3,233,946",드라마,12세관람가,일반영화,2014
4,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,"33,577,136,860","3,597,963",액션,12세관람가,일반영화,2019
5,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21,장편,미국,2648,114810421450,13747792,"28,824,682,580","3,225,974",애니메이션,전체관람가,일반영화,2019


len = 4835 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len = 4835 ['명량', '극한직업', '신과함께-죄와 벌', '국제시장', '어벤져스: 엔드게임', '겨울왕국 2', '아바타', '베테랑', '괴물', '도둑들']
len = 4835 ['김한민', '이병헌', '김용화', '윤제균', '안소니 루소', '제니퍼 리', '제임스 카메론', '류승완', '봉준호', '최동훈']
len = 4835 [2014, 2019, 2017, 2014, 2019, 2019, 2009, 2015, 2006, 2012]


## 3 함수 실행하기

### 3-1 변수 할당 및 실행

In [32]:
# id, secret 입력
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

# 함수 실행
# MovieAPI 클래스를 ma 변수에 할당
ma = MovieAPI()
# 수집된 전체 df와 중복 제거한 df를 각각의 변수에 할당, mk_movie_df 함수 실행
nv_df, nv_df_drop_dp = ma.mk_movie_df(index_list=index_list, movie_list=movie_list, dir_list=dir_list, year_list=year_list, client_id=client_id, client_secret=client_secret)

display(nv_df.head())

10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
110번째 완료
120번째 완료
130번째 완료
140번째 완료
150번째 완료
160번째 완료
170번째 완료
180번째 완료
190번째 완료
200번째 완료
210번째 완료
220번째 완료
230번째 완료
240번째 완료
250번째 완료
260번째 완료
270번째 완료
280번째 완료
290번째 완료
300번째 완료
310번째 완료
320번째 완료
330번째 완료
340번째 완료
350번째 완료
360번째 완료
370번째 완료
380번째 완료
390번째 완료
400번째 완료
410번째 완료
420번째 완료
430번째 완료
440번째 완료
450번째 완료
460번째 완료
470번째 완료
480번째 완료
490번째 완료
500번째 완료
savepoint 500
510번째 완료
520번째 완료
530번째 완료
540번째 완료
550번째 완료
560번째 완료
570번째 완료
580번째 완료
590번째 완료
600번째 완료
610번째 완료
620번째 완료
630번째 완료
640번째 완료
650번째 완료
660번째 완료
670번째 완료
680번째 완료
690번째 완료
700번째 완료
710번째 완료
720번째 완료
730번째 완료
740번째 완료
750번째 완료
760번째 완료
770번째 완료
780번째 완료
790번째 완료
800번째 완료
810번째 완료
820번째 완료
830번째 완료
840번째 완료
850번째 완료
860번째 완료
870번째 완료
880번째 완료
890번째 완료
900번째 완료
910번째 완료
920번째 완료
930번째 완료
940번째 완료
950번째 완료
960번째 완료
970번째 완료
980번째 완료
990번째 완료
1000번째 완료
savepoint 1000
1010번째 완료
1020번째 완료
1030번째 완료
1040번째 완료
1050번째 완료
1060번째 완료
1070번째 완료
1080번째 완료

c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
0,명량,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/09...,2014,[김한민],"[최민식, 류승룡, 조진웅]",8.44
1,극한직업,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/16...,2018,[이병헌],"[류승룡, 이하늬, 진선규, 이동휘, 공명]",8.51
2,신과함께-죄와 벌,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/08...,2017,[김용화],"[하정우, 차태현, 주지훈, 김향기, 김동욱, 마동석]",7.83
3,국제시장,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,2014,[윤제균],"[황정민, 김윤진, 오달수, 정진영]",9.02
4,어벤져스: 엔드게임,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/13...,2019,"[앤서니 루소, 조 루소]","[로버트 다우니 주니어, 크리스 에반스, 크리스 헴스워스, 마크 러팔로, 스칼릿 조...",9.38


In [26]:
movie_df[movie_df.index == 421] # 개빡치네^^...

,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
421,사랑,곽경택,롯데쇼핑㈜롯데엔터테인먼트,2007-09-19,장편,한국,0,0,2123815,0,"547,926",멜로/로맨스,15세관람가,일반영화,2007


### 3-2 누락 갯수 확인하기

In [46]:
len(movie_df) - len(nv_df)

337

### 3-3 누락 리스트 확인하기

In [48]:
import numpy as np

aa = nv_df.index.to_numpy()             # 추출 인덱스 (ndarray 1차원)
bb = np.array(range(len(movie_df)))     # 원본 인덱스 (ndarray 1차원)

diff_index = np.setdiff1d(bb, aa)       # 원본 인덱스(bb)에서 추출 인덱스(aa) 빼기
missed_df = movie_df.loc[diff_index]    # 원본 파일(movie_df)에서 해당 인덱스의 DataFrame 뽑아내기

missed_df.to_csv('./files/missed_utf8.csv', encoding='utf-8')
print('누락리스트',len(missed_df))
missed_df.head()

411


,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
156,강철중: 공공의 적 1-1,강우석,(주)씨제이이엔엠,2008-06-19,장편,한국,659,28465757767,4300670,"9,048,970,100","1,298,197",드라마,15세관람가,일반영화,2008
253,하모니,강대규,(주)씨제이이엔엠,2010-01-28,장편,한국,405,21992266198,3068544,"5,451,128,000","718,993",드라마,12세관람가,일반영화,2010
329,보안관,김형주,롯데쇼핑㈜롯데엔터테인먼트,2017-05-03,장편,한국,1091,20697532509,2571491,"4,093,422,209","491,530",범죄,15세관람가,일반영화,2017
421,사랑,곽경택,롯데쇼핑㈜롯데엔터테인먼트,2007-09-19,장편,한국,0,0,2123815,0,"547,926",멜로/로맨스,15세관람가,일반영화,2007
444,소울,피트 닥터,월트디즈니컴퍼니코리아 유한책임회사,2021-01-20,장편,미국,2018,19034164180,2048137,"6,297,246,010","659,789",애니메이션,전체관람가,일반영화,2021


In [50]:
error_list = pd.read_csv('./files/error_list_4500_utf8.csv', encoding='utf-8', index_col=0)
nofind_index_li = error_list.loc[error_list['1']=='검색 불가']['0']
nofind_df = movie_df.loc[nofind_index_li]
print('검색불가 리스트', len(nofind_df))
nofind_df.head()

275


,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
156,강철중: 공공의 적 1-1,강우석,(주)씨제이이엔엠,2008-06-19,장편,한국,659,28465757767,4300670,"9,048,970,100","1,298,197",드라마,15세관람가,일반영화,2008
458,육사오(6/45),박규태,(주)홈초이스,2022-08-24,장편,한국,1159,19791166880,1980769,"3,932,407,651","385,835",코미디,12세관람가,일반영화,2022
672,해적: 도깨비 깃발,김정훈,롯데컬처웍스(주)롯데엔터테인먼트,2022-01-26,장편,한국,1708,12523963990,1339242,"2,843,506,630","319,433",어드벤처,12세관람가,일반영화,2022
752,인생은 아름다워,최국희,롯데컬처웍스(주)롯데엔터테인먼트,2022-09-28,장편,한국,1285,10898590019,1171648,"2,831,607,046","298,657",뮤지컬,12세관람가,일반영화,2022
772,"10,000 BC",롤랜드 에머리히,워너브러더스 코리아(주),2008-03-13,장편,미국,359,7430607499,1140118,"2,251,461,500","326,091",어드벤처,15세관람가,일반영화,2008


### 3-4 누락 리스트 다시 돌리기

# ▼ 아래는 함수 오류 확인용

### 0 준비하기

In [95]:
import pandas as pd

# 리스트 불러오기
missed_df = pd.read_csv('./files/missed_utf8.csv', encoding='utf-8', index_col=0)
display(missed_df[:6])
index_list_m = list(missed_df.index)
movie_list_m = missed_df['영화명'].to_list()
dir_list_m = missed_df['감독'].to_list()
year_list_m = missed_df['연도'].to_list()

print('len =',len(index_list_m), index_list[:10])
print('len =',len(movie_list_m), movie_list[:10])
print('len =',len(dir_list_m), dir_list[:10])
print('len =',len(year_list_m), year_list[:10])

,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,연도
156,강철중: 공공의 적 1-1,강우석,(주)씨제이이엔엠,2008-06-19,장편,한국,659,28465757767,4300670,"9,048,970,100","1,298,197",드라마,15세관람가,일반영화,2008
253,하모니,강대규,(주)씨제이이엔엠,2010-01-28,장편,한국,405,21992266198,3068544,"5,451,128,000","718,993",드라마,12세관람가,일반영화,2010
329,보안관,김형주,롯데쇼핑㈜롯데엔터테인먼트,2017-05-03,장편,한국,1091,20697532509,2571491,"4,093,422,209","491,530",범죄,15세관람가,일반영화,2017
421,사랑,곽경택,롯데쇼핑㈜롯데엔터테인먼트,2007-09-19,장편,한국,0,0,2123815,0,"547,926",멜로/로맨스,15세관람가,일반영화,2007
444,소울,피트 닥터,월트디즈니컴퍼니코리아 유한책임회사,2021-01-20,장편,미국,2018,19034164180,2048137,"6,297,246,010","659,789",애니메이션,전체관람가,일반영화,2021
458,육사오(6/45),박규태,(주)홈초이스,2022-08-24,장편,한국,1159,19791166880,1980769,"3,932,407,651","385,835",코미디,12세관람가,일반영화,2022


len = 411 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len = 411 ['데자뷰']
len = 411 ['토니 스콧']
len = 411 [2007]


### 1 xml 형태 확인

In [91]:
dir_list_m[13]

'토니 스콧'

In [101]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

import urllib.request
import xmltodict
import pandas as pd
import time
import re

start_n = 1
end_n = 2

index = index_list_m[start_n:end_n]
year_list = year_list_m[start_n:end_n]
dir_list = dir_list_m[start_n:end_n]
movie_list = movie_list_m[start_n:end_n]

for i, movie_o in enumerate(movie_list):
    movie = movie_o.replace('(애니)','').replace('(다큐)','').replace('감독판','').replace('확장판','').replace('X2','엑스 2').replace('(6/45)','')
    movie = re.sub('[^가-힣A-Za-z0-9\s,-]', '', movie)   # 한글, 영어, 숫자, 공백 빼고 지우기
    encText = urllib.parse.quote(movie)
    params = f'"&display=100&start=1&yearfrom={str(int(year_list[i])-2)}&yearto={str(int(year_list[i]))}'
    url = 'https://openapi.naver.com/v1/search/movie.xml?query="' + encText  + params
    time.sleep(0.5)
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
        search_li = dict_data['rss']['channel']
        display(search_li)
        if int(search_li['total']) != 0:
            # 검색 결과 있는 경우 
            search_li = search_li['item']
            if type(search_li) == list:
                # 검색 결과가 여러개인 경우
                for j in range(len(search_li)):
                    mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
                    mv_title = re.sub('[^가-힣A-Za-z0-9\s,-]', '', mv_title)
                    mv_rating = search_li[j]['userRating']
                    mv_dir = search_li[j]['director']
                    mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                    mv_actor = search_li[j]['actor']
                    mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                    mv_year = search_li[j]['pubDate']
                    if mv_title.replace(' ','') == movie.replace(' ','') and mv_rating != None and mv_dir != None and mv_actor != None:
                        # 영화명 같고, 평점-감독-배우 있으면,
                        print(mv_rating)
                        if dir_list[i] in mv_dir:
                            # 평점이 0.00이 아니고, 감독명이 API에서 찾은 리스트 있으면
                            print(index_list_m[i],
                            movie_o,
                            search_li[j]['link'],
                            search_li[j]['image'],
                            mv_year,
                            mv_dir,
                            mv_actor,
                            mv_rating)
                            break   # 뒤에 더 탐색할 필요가 없음
                        else:
                            print('다른문제1')
                            break
                    else: print('다른문제2')
            elif type(search_li) == dict:
                # 검색 결과가 하나인 경우
                mv_title = search_li['title'].replace('</b>','').replace('<b>','')
                mv_dir = search_li['director']
                mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                mv_actor = search_li['actor']
                mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

                print(index_list_m[i],
                            movie_o,
                            search_li['link'],
                            search_li['image'],
                            mv_year,
                            mv_dir,
                            mv_actor,
                            mv_rating)

{'title': "Naver Open API - movie ::'&quot;하모니&quot;'",
 'link': 'https://search.naver.com',
 'description': 'Naver Search Result',
 'lastBuildDate': 'Mon, 27 Mar 2023 03:13:18 +0900',
 'total': '6',
 'start': '1',
 'display': '6',
 'item': [{'title': '<b>하모니</b>',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=88170',
   'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0881/88170_P01_163200.jpg',
   'subtitle': 'Harmony',
   'pubDate': '2010',
   'director': '이올림|',
   'actor': None,
   'userRating': '8.20'},
  {'title': '해피<b>하모니</b> 다마고치',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=133254',
   'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1332/133254_P01_161426.jpg',
   'subtitle': 'たまごっち!.',
   'pubDate': '2010',
   'director': '시무라 조지|',
   'actor': '양정화|박영남|김민정|고구인|조경이|박고운|채민지|문유정|김혜성|신경선|김연우|강시현|이인석|정주원|',
   'userRating': '0.00'},
  {'title': '<b>하모니</b>',
   'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=53034',
   

다른문제2
다른문제2
9.05
156 하모니 https://movie.naver.com/movie/bi/mi/basic.nhn?code=53034 https://ssl.pstatic.net/imgmovie/mdi/mit110/0530/53034_P00_113135.jpg 2009 ['강대규'] ['김윤진', '나문희', '강예원', '이다희', '장영남', '박준면', '정수영'] 9.05


### 2 함수 실행해서 확인

In [97]:
# id, secret 입력
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

# 함수 실행
# MovieAPI 클래스를 ma 변수에 할당
ma = MovieAPI()
# 수집된 전체 df와 중복 제거한 df를 각각의 변수에 할당, mk_movie_df 함수 실행
nv_df_m, nv_df_m_drop_dp = ma.mk_movie_df(index_list=index_list_m, movie_list=movie_list_m, dir_list=dir_list_m, year_list=year_list_m, client_id=client_id, client_secret=client_secret)

display(nv_df_m.head())

421번째 완료
486번째 완료
752번째 완료
1251번째 완료
1545번째 완료
1758번째 완료
1870번째 완료
2024번째 완료
2195번째 완료
2337번째 완료
2460번째 완료
2589번째 완료
2823번째 완료
2956번째 완료
3109번째 완료
3173번째 완료
3184번째 완료
3300번째 완료
3362번째 완료
3438번째 완료
3521번째 완료
3590번째 완료
3649번째 완료
3721번째 완료
3765번째 완료
3846번째 완료
3916번째 완료
3966번째 완료
4014번째 완료
4020번째 완료
4027번째 완료
4093번째 완료
4155번째 완료
4206번째 완료
4263번째 완료
4330번째 완료
4387번째 완료
4442번째 완료
4489번째 완료
4543번째 완료
4607번째 완료
4648번째 완료
4686번째 완료
4722번째 완료
4755번째 완료
4821번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,


In [100]:
display(nv_df_m.head())

,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
156,강철중: 공공의 적 1-1,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/06...,2008,[강우석],"[설경구, 정재영]",8.53
2338,명탐정 코난: 에피소드 원-작아진 명탐정,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/15...,2016,[야마모토 야스이치로],"[야마구치 캇페이, 타카야마 미나미, 야마자키 와카나, 하야시바라 메구미, 코야마 ...",6.42
3343,샤크보이와 라바걸의 모험 : 3-D,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,2005,[로버트 로드리게즈],"[테일러 로트너, 테일러 둘리]",7.50
4019,D-day-어느날 갑자기 세번째 이야기,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/06...,2006,[김은경],"[김리나, 이은성, 유호린, 허진용]",7.67
4345,나루토-대흥분! 초승달 섬의 애니멀 소동,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/03...,2006,[츠루 토시유키],"[다케우치 준코, 나카무라 치에, 이노우에 카즈히코, 마스카와 요이치]",7.41


In [ ]:
# https://jimmy-ai.tistory.com/102
# df.drop_duplicates()

# ⊙ 맨 아래